# Visualizing ARC Tasks

JaxARC offers two complementary visualization methods:

1. **SVG Visualization** - High-quality vector graphics ideal for:
   - Jupyter notebooks and IPython
   - Documentation and presentations
   - Publication-quality figures

2. **Rich Terminal Output** - Colored console output perfect for:
   - Interactive development
   - Quick debugging
   - Terminal-based workflows

Both methods work with the same data structures and produce beautiful, color-accurate representations of ARC grids.

In [ ]:
# Import required libraries
import jax
from jaxarc.parsers.arc_agi import ArcAgiParser
from jaxarc.configs import DatasetConfig
from jaxarc.utils.core import get_config

# Load configuration
cfg = get_config()
dataset_config = DatasetConfig.from_hydra(cfg.dataset)

# Initialize parser
parser = ArcAgiParser(dataset_config)

## Setup: Loading a Task

First, let's load a random ARC task using the parser. We'll use this task throughout the tutorial to demonstrate different visualization techniques.

In [ ]:
# Get a random task (using seed 123 for reproducibility)
key = jax.random.PRNGKey(123)
task = parser.get_random_task(key)

# Display task information
print(f"Task Index: {task.task_index}")
print(f"Training Pairs: {task.num_train_pairs}")
print(f"Test Pairs: {task.num_test_pairs}")

## Method 1: Single Grid Visualization

Let's start by visualizing individual grids. This is useful when you want to inspect a specific input or output grid in detail.

In [ ]:
# Import visualization functions
from IPython.display import display
from jaxarc.utils.visualization import draw_grid_svg, visualize_grid_rich

# Get the first training pair
task_pair = task.get_train_pair(0)
input_grid = task_pair.input_grid
output_grid = task_pair.output_grid

print(f"Input grid shape: {input_grid.shape}")
print(f"Output grid shape: {output_grid.shape}")
print()

# SVG visualization - high quality for notebooks
print("SVG Visualization (Input):")
input_svg = draw_grid_svg(input_grid, label="Training Input")
display(input_svg)


In [ ]:

print("\nSVG Visualization (Output):")
output_svg = draw_grid_svg(output_grid, label="Training Output")
display(output_svg)


In [ ]:

# Rich terminal visualization - great for quick debugging
print("\nRich Terminal Output (Input):")
display(visualize_grid_rich(input_grid, title="Training Input (Rich)"))


In [ ]:

print("\nRich Terminal Output (Output):")
display(visualize_grid_rich(output_grid, title="Training Output (Rich)"))

## Method 2: Input/Output Pair Visualization

Now let's visualize the transformation from input to output. This side-by-side view makes it easy to see the pattern or rule that transforms the input into the output. **Arrow indicator** shows the transformation direction

In [ ]:
# Import pair visualization functions
from jaxarc.utils.visualization import draw_task_pair_svg, visualize_task_pair_rich

# SVG pair visualization - shows transformation clearly
print("🎨 SVG Pair Visualization:")
pair_svg = draw_task_pair_svg(
    input_grid=input_grid,
    output_grid=output_grid,
    label="Training Pair 1",
)
display(pair_svg)


In [ ]:

# Rich terminal pair visualization
print("\n💻 Rich Terminal Pair Output:")
visualize_task_pair_rich(
    input_grid=input_grid,
    output_grid=output_grid,
    title="Training Pair 1 (Rich)",
)

## Method 3: Complete Task Visualization

For understanding the full task, we can visualize all training examples together. This comprehensive view helps identify patterns across multiple examples.

### Visualization Options

- `include_test=False` - Show only training pairs (outputs known)
- `include_test=True` - Include test inputs with "?" for unknown outputs
- `include_test="all"` - Show everything including test outputs (if available)

In [ ]:
# Import complete task visualization functions
from jaxarc.utils.visualization import draw_parsed_task_data_svg, visualize_parsed_task_data_rich

# Option 1: Training examples only
print("🎨 Complete Task - Training Only:")
task_svg = draw_parsed_task_data_svg(
    task_data=task,
    include_test=False,  # Show only training pairs
)
display(task_svg)


In [ ]:

# Option 2: Include test inputs (outputs unknown)
print("\n🎨 Complete Task - With Test Input:")
task_svg_with_test = draw_parsed_task_data_svg(
    task_data=task,
    include_test=True,  # Show test inputs with "?" for outputs
)
display(task_svg_with_test)


In [ ]:

# Rich terminal version - compact overview
print("\n💻 Rich Terminal - Complete Task:")
visualize_parsed_task_data_rich(
    task_data=task,
    show_test=True,
    show_coordinates=False,
    show_numbers=False,
    double_width=True,
)

### Customization Options

All visualization functions support customization:

```python
# Grid customization
draw_grid_svg(grid, max_width=15.0, max_height=10.0, label="Custom")

# Show coordinates in terminal
visualize_grid_rich(grid, show_coordinates=True, show_numbers=True)

# Custom task layout
draw_parsed_task_data_svg(task, width=40.0, border_colors=["#0066cc", "#00cc66"])
```

## Iterating Through Training Examples

When analyzing a task, you often want to examine each training example individually. This loop demonstrates how to visualize multiple pairs programmatically.

### Practical Use Cases

- **Pattern recognition** - Study how the transformation rule applies across examples
- **Edge case detection** - Identify examples that behave differently
- **Agent debugging** - Compare your agent's output with expected results for each example

In [ ]:
# Iterate through training pairs (showing first 2 for documentation brevity)
from IPython.display import display

num_pairs_to_show = min(2, task.num_train_pairs)

for i in range(num_pairs_to_show):
    pair = task.get_train_pair(i)
    
    print(f"\n{'='*60}")
    print(f"Training Pair {i + 1} / {task.num_train_pairs}")
    print(f"{'='*60}")
    
    # Visualize this pair
    pair_svg = draw_task_pair_svg(
        input_grid=pair.input_grid,
        output_grid=pair.output_grid,
        label=f"Training Pair {i + 1}",
    )
    display(pair_svg)

# Show test input if available
if task.num_test_pairs > 0:
    print(f"\n{'='*60}")
    print(f"Test Input (Output Unknown)")
    print(f"{'='*60}")
    test_input = task.get_test_input_grid(0)
    display(draw_grid_svg(test_input, label="Test Input"))